`fixUnder` is a script to take the poems from EMR *Under the Tree* and put each in its own file in the folder `/texts/emr/UnderTree`. I hade used Notepad++ to insert a code in front of each poem title (#) in the source file. I also had to delete the empty lines and title at the top of the source file, and fix a small number of poems that had a subtitle ("A Song"), that is now on the same line as the title. 

5-25: New strategy: each poem is saved into a list and markup is entered based on the poem's consistent internal structure. I used the inserted # to trigger the end of the previous poem and the opening of a new list with the next line.  Once all the poems are marked up, we print them all at once. So now we can insert a header line, with poem count, <lg> tags with stanza counts, and <l> tags with a line count. The markup is TEI conformant, following the basic pattern at https://teibyexample.org/tutorials/TBED04v00.htm. I would like to figure out how to do the markup in JSON because that is more database-friendly, but this will do for now.



In [35]:
import re

file = "texts/emr/under.txt"

hashcount = 0    # for counting hashtags to trigger title printing
titlecount = 1   # for counting the poems in book order
linecount = 1    # for counting poem lines
poemlist = None  # for naming individual poem files
booklist = []    # for keeping each poem as a big list

with open (file, "r", encoding='utf-8-sig') as f:  #needed encoding to delete the BOM code
    for line in f:
        stripped = line.strip()
        if '#' in stripped :          
            if poemlist :
                booklist.append(poemlist)
            hashcount += 1
        if "#" not in stripped and hashcount == 1 :
            title = stripped
            poemlist = []
            # putting f before a string def allows vars in curly braces in string
            titlecountstring = f'n="{titlecount}">'
            poemlist.append(f'<lg xmlns="http://www.tei-c.org/ns/1.0" type="poem" {titlecountstring}')
            poemlist.append(title)
            hashcount = 0
            titlecount += 1
            linecount = 1
        if "#" not in stripped and stripped != title and hashcount == 0 :
            if stripped == "" :
                poemlist.append(stripped) # keep the empty line
            else :
                linecountstring = f'n="{linecount}">'
                poemlist.append(f'<l {linecountstring}{stripped}</l>')
                linecount += 1

for poem in booklist :
    poemfile_name = f'texts/emr/UnderTree/{poem[1]}.txt' # title is now the second item on the poemlist
    poem[1] = f'<head>{poem[1]}</head>' # alter title line by adding tags
        #stanzas have a blank line between them, so find the blank lines followed by text lines
    blanks = [index for index in range(len(poem)-1) if poem[index] == "" and poem[index+1] != ""]
    
        #for each blank line, insert the stanza tag with number
    for n, blank in enumerate(blanks) :
        if blank == 3 :
            poem[blank] = f'<lg type="stanza" n="{n+1}">'
        else :
            poem[blank] = f'</lg><lg type="stanza" n="{n+1}">'
        
        #put the end tags on the blank lines at the end
    poem[-3] = '</lg>'
    poem[-2] = '</lg>'
    
    #print the TEI version to its own file (or later, do more stuff to it)
    with open(poemfile_name, 'w') as p :
        for line in poem :
            print (line, file=p)

